In [1]:
# Establecemos los directorios del proyecto
import sys
from pathlib import Path
sys.path.append(str(Path('.').resolve().parent))

In [2]:
from langchain import PromptTemplate
# from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
#from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import base64
import os
from api.configuration.keys import POKER_GPT_KEY
from api.configuration.config import IMG_DIR


/home/y41000/repos/gpt-poker-player/api/images


In [3]:

# Almacenamos modelo
gptmodel = ChatOpenAI(model_name = "gpt-4o", openai_api_key=POKER_GPT_KEY)
# Mi username en la plataforma de poker
#user_poker_name="pokeados"

In [4]:
print(gptmodel.invoke('Hola').content)
print(gptmodel([HumanMessage(content="Hola")]).content)

¡Hola! ¿Cómo puedo ayudarte hoy?


/home/y41000/miniconda3/envs/poker/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


¡Hola! ¿Cómo estás? ¿En qué puedo ayudarte hoy?


CONSTRUIMOS EL TEMPLATE

In [66]:
# Función para codificar la imagen que le vamos a pasar a langchain

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


In [67]:
context_temp = """You are a profesional poker player called {user_poker_name}.
Current situation: We are in {stage}. Player {player} make a {play} from {seat} position.
Players statisticts: Player {player} have played {n_plays} hands with {n_raises} rises and {n_wins} wins.
See the table: {imagen}
Tell me: What move would you do and why."""

imagen = encode_image(f"{IMG_DIR}/Mano_2024-05-12:23-14-41.png")

# construimos el template
template = PromptTemplate(
    input_variables=['user_poker_name','stage','player','play','seat','player','n_plays','n_raises','n_wins','imagen'],
    template = context_temp)

# Vemos cómo sale el prompt
prompt_test = template.format(user_poker_name='pokeados',
                              player='HER_MILK_MAN',
                              stage='preflop',
                              seat='UTG',
                              play = 'call',
                              n_plays = 33,
                              n_raises = 25,
                              n_wins = 2,imagen=imagen)
#print(prompt_test)

Así a lo bruto no es posible procesar la información porque la imagen arrasa con el límite de tokens.
Vamos a usar las cadenas de LangChain.

In [50]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.llms import OpenAI
from langchain.schema import HumanMessage
import matplotlib
matplotlib.use('TkAgg')  # Usar la backend TkAgg, que es interactiva
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



In [11]:
# Función para codificar la imagen
def encode_image(image_path):
    with open(image_path, 'rb') as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')
    
base64_image = encode_image(f"{image_path}{im}")

In [75]:
# PRIMERA CADENA
llm = ChatOpenAI(model_name="gpt-4o", openai_api_key=POKER_GPT_KEY)

# En la primera cadena vamos a configurar la imagen que le vamos a enviar a la segunda cadena
# Imagen
image_path = f"{IMG_DIR}/Mano_2024-05-12:23-13-25.png"
base64_image = encode_image(image_path)
# Componente LangChain
infromacion_mesa = HumanMessage(
    content=[
        {
            "type": "text", "text": "Extract all the relevant information of the table, unambiguous."
        },
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        }
    ]
)

infromacion_mesa =PromptTemplate.from_template(llm.invoke([infromacion_mesa]).content)


# Creamos la cadena de LangChain con el LLM y el prompt de la mesa de poker
chain1 = LLMChain(llm=llm, prompt=infromacion_mesa, output_key="Current_situation") # Estos dos parámetros son los más importantes para la cadena





In [76]:
# SEGUNDA CADENA

# construimos el template donde le damos el contexto de los jugadores

context_temp = """You are a profesional poker player called {user_poker_name}.
Current situation: We are in {stage}. Player {player} make a {play} from {seat} position.
Players statisticts: Player {player} have played {n_plays} hands with {n_raises} rises and {n_wins} wins.
This is the current situation of the table:{Current_situation}
Tell me: What move would you do. Just tell me the play, and why, in two diferents bullets."""

template = PromptTemplate(
    input_variables=['user_poker_name','stage','player','play','seat','player','n_plays','n_raises','n_wins'],
    template = context_temp)

# Creamos la cadena de LangChain con el LLM y el template
chain2 = LLMChain(llm=llm, prompt=template, output_key="Optiomal_move") # Estos dos parámetros son los más importantes para la cadena




In [77]:
# CONCATENACIÓN DE CADENAS

final_chain = SequentialChain(
    chains=[chain1,chain2],
    input_variables=['user_poker_name','player','stage','seat','play','n_plays','n_raises','n_wins'],
    output_variables=['Current_situation','Optiomal_move'],
    verbose=True
)


In [78]:
final_chain({'user_poker_name':'pokeados',
             'player':'HER_MILK_MAN',
             'stage':'preflop',
             'seat':'UTG',
             'play':'call',
             'n_plays':33,
             'n_raises':25,
             'n_wins':2}
)



> Entering new SequentialChain chain...

> Finished chain.


{'user_poker_name': 'pokeados',
 'player': 'HER_MILK_MAN',
 'stage': 'preflop',
 'seat': 'UTG',
 'play': 'call',
 'n_plays': 33,
 'n_raises': 25,
 'n_wins': 2,
 'Current_situation': "Based on the provided information, here is a strategic analysis and some advice for the player, likely pokeados:\n\n### Current Situation\n- **Player (pokeados) Cards**: 8♣ and 6♣\n- **Player Chip Count**: 2,384 chips\n- **Current Pot**: 250 chips\n- **Blinds**: 100 (big blind) and 50 (small blind)\n- **Player Position**: Not explicitly mentioned, but likely in early position given the options.\n\n### Player Options\n1. **Ausentarme en la próxima mano (Sit out next hand)**: This might be useful if you need a break but not recommended if you want to actively play and improve your chip stack.\n2. **Ausentarme en la próxima ciega grande (Sit out next big blind)**: Useful if you plan to leave the table soon, but not relevant to immediate hand strategy.\n3. **Retirarse si hay apuesta (Fold if there is a bet)**:

In [79]:
imag = mpimg.imread(image_path)
plt.imshow(imag)
plt.axis('off')  # Ocultar los ejes
plt.show()


In [ ]:


# Ejecutamos la cadena con los datos
result = chain.run(
   { user_poker_name='pokeados',
    player='HER_MILK_MAN',
    stage='preflop',
    seat='UTG',
    play = 'call',
    n_plays = 33,
    n_raises = 25,
    n_wins = 2},
    imagen=f"data:image/png;base64,{base64_image}"
)

In [68]:
# Pasémosle el prompt al modelo
respuesta = gptmodel.invoke(prompt_test)
print(respuesta.content)

Given the information provided, there are several factors to consider before making a decision:

1. **Player Statistics:** HER_MILK_MAN has played 33 hands with 25 raises and only 2 wins. This suggests that HER_MILK_MAN is an extremely aggressive player (raising around 75% of the hands), but with a low success rate. This could indicate that they are often playing weak hands or bluffing.

2. **Position:** HER_MILK_MAN is calling from the UTG (Under The Gun) position, which is typically a strong position, as players are usually more cautious and play stronger hands from here. However, given their aggressive and loose statistics, this call may not necessarily indicate strength.

3. **Your Position and Hand Strength:** Your decision should also be influenced by your position (are you in early, middle, or late position?) and the strength of your hand. Without specific information on your hand, I can provide a general strategy.

### Potential Moves:

- **Raise:** If you have a strong hand (e

In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import base64

# Función para codificar la imagen
def encode_image(image_path):
    with open(image_path, 'rb') as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# Imagen
image_path = f"{IMG_DIR}/Mano_2024-05-12:23-14-41.png"
base64_image = encode_image(image_path)

# Define la plantilla del prompt para describir la imagen
prompt_text = """Extract all relevant information from the table, without ambiguities, to pass to the next chain.
Image data: {image_data}"""

# Crear el template
template = PromptTemplate(
    input_variables=['image_data'],
    template=prompt_text
)

# Crear la cadena de LangChain para describir la imagen
llm = ChatOpenAI(model_name="gpt-4o", openai_api_key=POKER_GPT_KEY)
chain1 = LLMChain(llm=llm, prompt=template)

# Ejecutar la cadena para obtener la descripción de la imagen
description = chain1.run(image_data=f"data:image/jpeg;base64,{base64_image}")

print(description)


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-vOpN9RsS0uOfo4bglanlWYYS on tokens per min (TPM): Limit 30000, Requested 146256. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}